In [1]:
!pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.9/389.9 kB 7.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5


In [2]:
# Import libraries
import openai

# Set your OpenAI API key
from google.colab import userdata
lm_client = openai.OpenAI(api_key = userdata.get('OPENAI_API_KEY'))  # Replace with your OpenAI key

In [5]:
import json

# Define the tools (functions) for support ticket management
tools = [
    {
        "name": "ticket_manager",
        "description": "Handles support tickets by identifying the appropriate action and extracting necessary details.",
        "parameters": {
            "type": "object",
            "strict": True,
            "properties": {
                "action": {
                    "type": "string",
                    "enum": ["create", "update", "close", "view"],
                    "description": "The action to take on a support ticket."
                },
                "team": {
                    "type": "string",
                    "enum": ["technical", "billing", "general", "product"],
                    "description": "The team responsible for addressing the ticket."
                },
                "ticket_details": {
                    "type": "object",
                    "description": "Details of the ticket, such as issue description, ticket ID, or priority.",
                }
            },
            "required": ["action", "team"],
        },
    }
]

In [6]:

# Define the LLM router for support ticket management
def support_ticket_router_llm(lm_client, user_query):
    system_message = '''
    Support Ticket Routing System

    Overview:
    This system processes customer support queries and routes them to the appropriate team or action. The output includes the action, team, and relevant ticket details.

    Actions:
    - create: Create a new support ticket.
    - update: Update an existing ticket with new details.
    - close: Close a resolved ticket.
    - view: Retrieve ticket status or details.

    Teams:
    - technical: Handles issues like software bugs or system errors.
    - billing: Deals with payment issues, refunds, or invoices.
    - general: General inquiries or non-specific support needs.
    - product: Questions about product features, usage, or documentation.

    Query Examples:
    1. "I need help with a billing issue; my payment failed."
       Output:
       {
           "action": "create",
           "team": "billing",
           "ticket_details": {"description": "Payment failed"}
       }

    2. "Update my technical issue ticket #12345 with the error logs."
       Output:
       {
           "action": "update",
           "team": "technical",
           "ticket_details": {"ticket_id": "12345", "update_info": "Error logs attached"}
       }

    3. "Close the ticket #67890 as the issue is resolved."
       Output:
       {
           "action": "close",
           "team": "general",
           "ticket_details": {"ticket_id": "67890"}
       }

    4. "Check the status of my product question, ticket #54321."
       Output:
       {
           "action": "view",
           "team": "product",
           "ticket_details": {"ticket_id": "54321"}
       }

    Instructions:
    Detect the action and team from the user query, and extract additional details as needed. Ensure the output is structured as JSON.
    '''

    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_query},
    ]

    response = lm_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        max_tokens=4000,
        temperature=0.2,
        functions=tools,
        function_call={"name": "ticket_manager"},
    )

    # Parse response and extract details
    arguments = json.loads(response.choices[0].message.function_call.arguments)

    action = arguments['action']
    team = arguments['team']
    ticket_details = arguments.get('ticket_details', {})
    return action, team, ticket_details

In [7]:
# Example usage
user_query = "I need to create a technical ticket about a system crash."
action, team, ticket_details = support_ticket_router_llm(lm_client, user_query)

print(f"Action: {action}, Team: {team}, Ticket Details: {ticket_details}")

Action: create, Team: technical, Ticket Details: {}
